### 반복량 축소

In [62]:
import pandas as pd
import numpy as np
import random
import time
from tqdm import tqdm


qnt = 10000 # 데이터 수

df_a_origin = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "some_data" : [random.randint(0, qnt) if random.randint(0, qnt) < qnt*4/5 else None
                   for _ in range(qnt)]
})

df_b = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "alt_data" : [random.randint(0, qnt) for _ in range(qnt)]
})

# 대조군 : data_a 의 결측 데이터를 data_b의 데이터로 치환
# O(N)의 속도를 보이며, 이는 아래 작업을 매번 반복한다.
# df_b["user_id"] == row["user_id"] → N개 비교 연산
# boolean mask 생성 → 메모리 비용
# loc으로 필터링 → N개 중에서 찾기
# 즉, 전체 복잡도는 거의 O(N*M) 구조가 됩니다.
df_a = df_a_origin.copy()
start = time.time()
for i, row in tqdm(df_a.iterrows()):
    if pd.isna(row["some_data"]):
        df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 1 ::: {time.time() - start}")

# 실험군 1 : None 인 idx를 미리 뽑아놓기 (횟수 감소)  
# 기존은 qnt 개를 for문 반복을 해야했다면
# 실험군 1에서는 None 개수만큼만 반복하므로 연산량 감소
df_a = df_a_origin.copy()
start = time.time()
target_idx = df_a[df_a["some_data"].isna()].index
for i, row in tqdm(df_a.loc[target_idx].iterrows()):
    df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 2 ::: {time.time() - start}")


10000it [00:01, 5924.72it/s]


time check 1 ::: 1.6888391971588135


2029it [00:01, 1458.36it/s]

time check 2 ::: 1.394446849822998


### 딕셔너리 캐싱  

In [64]:
import pandas as pd
import numpy as np
import random
import time
from tqdm import tqdm


qnt = 10000 # 데이터 수

df_a_origin = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "some_data" : [random.randint(0, qnt) if random.randint(0, qnt) < qnt*4/5 else None
                   for _ in range(qnt)]
})

df_b = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "alt_data" : [random.randint(0, qnt) for _ in range(qnt)]
})

# 대조군 : data_a 의 결측 데이터를 data_b의 데이터로 치환
df_a = df_a_origin.copy()
start = time.time()
for i, row in tqdm(df_a.iterrows()):
    if pd.isna(row["some_data"]):
        df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 1 ::: {time.time() - start}")

# 실험군 1 : 딕셔너리 캐싱  
# O(n) -> O(1)
# df_b["user_id"] == ... 처럼 user_id를 찾는 연산은 O(n)
# 딕셔너리는 key 기반으로 해시테이블에 정리가 되어있음 -> O(1)의 시간복잡도를 가짐
# (1) 캐싱 : 자주 쓰는 값을 미리 준비해두는 것  
# (2) 딕셔너리 해시 : 내부적으로 해시(hash) 라는 알고리즘을 사용해 key를 숫자로 바꾸고, 그 숫자를 이용해 저장 위치를 바로 찾음
# (3) 캐싱 = "필요한 값을 미리 저장해두는 행위", 딕셔너리 해시 = "미리 지정된 값들에 O(1)로 접근하게 해주는 데이터 구조"
df_a = df_a_origin.copy()
start = time.time()
b_dict = {uid : row for uid, row in df_b.set_index("user_id").iterrows()}
for i, row in tqdm(df_a.iterrows()):
    df_a.at[i, "some_data"] = b_dict[row["user_id"]]["alt_data"]
print(f"time check 2 ::: {time.time() - start}")

# 실험군 2 : 딕셔너리 캐싱 + 반복량 축소
df_a = df_a_origin.copy()
start = time.time()
target_idx = df_a[df_a["some_data"].isna()].index
b_dict = {uid : row for uid, row in df_b.set_index("user_id").iterrows()}
for i, row in tqdm(df_a.iloc[target_idx].iterrows()):
    df_a.at[i, "some_data"] = b_dict[row["user_id"]]["alt_data"]
print(f"time check 3 ::: {time.time() - start}")


10000it [00:01, 5900.42it/s]


time check 1 ::: 1.6957197189331055


10000it [00:00, 19939.96it/s]


time check 2 ::: 0.6568958759307861


2037it [00:00, 20414.99it/s]

time check 3 ::: 0.34510254859924316


In [65]:
# 구체적으로 뜯어보기 - 기존방식
df_a = df_a_origin.copy()
start = time.time()
# (1) df_a를 돌면서 -> 비용 = O(N_a)
for i, row in tqdm(df_a.iterrows()):
    if pd.isna(row["some_data"]):
        # (2) df_b를 돌면서 user_id를 찾음 -> 비용 = O(N_b)
        df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 1 ::: {time.time() - start}")
##### ==> 총 비용 : O(N_a) * O(N_b) / a, b 대략 1만건 씩이면 = 총 1억회 연산


# 구체적으로 뜯어보기 1
df_a = df_a_origin.copy()
time_check = time.time()
# (1) df_b 를 돌면서 dict를 만든다 -> 비용 = O(N_b)
b_dict = {uid : row for uid, row in df_b.set_index("user_id").iterrows()}
print(f"time check 1 ::: {time.time() - time_check}")
# (2) df_a 를 돌면서 -> 비용 = O(N_a)
time_check = time.time()
for i, row in tqdm(df_a.iterrows()):
    # (3) 해시테이블에 바로 접근해서 값을 가져옴 -> 비용 = O(1) 에 근접
    df_a.at[i, "some_data"] = b_dict[row["user_id"]]["alt_data"]
print(f"time check 2 ::: {time.time() - time_check}")
##### ==> 총 비용 : O(N_b) + O(N_a) * O(1) / a, b 대략 1만건 씩이면 = 총 20000 회 연산 (에 근접)


# 구체적으로 뜯어보기 2
df_a = df_a_origin.copy()
time_check = time.time()
# (1) df 를 돌면서 dict를 만든다 -> 비용 = O(N_b)
b_dict = {uid : row for uid, row in df_b.set_index("user_id").iterrows()}
# (2) df 를 돌면서 na 인덱스 식별 -> 비용 = O(N_a) 
target_idx = df_a[df_a["some_data"].isna()].index
print(f"time check 1 ::: {time.time() - time_check}")
time_check = time.time()
# (3) 필터링된 df 를 돌면서 -> 비용 = O(N_a * 0.2)  // 위에서 NULL 비율을 대략 0.2로 잡음
for i, row in tqdm(df_a.iloc[target_idx].iterrows()):
    # (4) 해시테이블에 바로 접근해서 값을 가져옴 -> 비용 = O(1) 에 근접
    df_a.at[i, "some_data"] = b_dict[row["user_id"]]["alt_data"]
print(f"time check 2 ::: {time.time() - time_check}")
##### ==> 총 비용 : O(N_b) + O(N_a)  +  O(N_a*0.2) * O(1) / a, b 대략 1만건 씩이면 = 총 22000회 연산
##### 그런데 왜 2번 방식보다 빠르냐? 이것은 바로..!
##### na 를 식별하는 isna() 연산이 "벡터연산"으로 굉장히 빠르기 때문

10000it [00:01, 6133.02it/s]


time check 1 ::: 1.6339082717895508
time check 1 ::: 0.283344030380249


10000it [00:00, 18258.66it/s]


time check 2 ::: 0.5476853847503662
time check 1 ::: 0.2526881694793701


2037it [00:00, 18197.07it/s]

time check 2 ::: 0.11291956901550293


### 해시 기반 join을 활용한 반복문 제거와 데이터프레임 연산 성능 최적화

In [56]:
import pandas as pd
import numpy as np
import random
import time
from tqdm import tqdm


qnt = 10000 # 데이터 수

df_a_origin = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "some_data" : [random.randint(0, qnt) if random.randint(0, qnt) < qnt*4/5 else None
                   for _ in range(qnt)]
})

df_b = pd.DataFrame({
    "user_id" : [str(x) for x in range(1, qnt+1)],
    "alt_data" : [random.randint(0, qnt) for _ in range(qnt)]
})

# 대조군 : data_a 의 결측 데이터를 data_b의 데이터로 치환
# O(N)의 속도를 보이며, 이는 아래 작업을 매번 반복한다.
# df_b["user_id"] == row["user_id"] → N개 비교 연산
# boolean mask 생성 → 메모리 비용
# loc으로 필터링 → N개 중에서 찾기
# 즉, 전체 복잡도는 거의 O(N*M) 구조가 됩니다.
df_a = df_a_origin.copy()
start = time.time()
for i, row in tqdm(df_a.iterrows()):
    if pd.isna(row["some_data"]):
        df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 1 ::: {time.time() - start}")

# 실험군 1 : None 인 idx를 미리 뽑아놓기 (횟수 감소)  
# 기존은 qnt 개를 for문 반복을 해야했다면
# 실험군 1에서는 None 개수만큼만 반복하므로 연산량 감소
df_a = df_a_origin.copy()
start = time.time()
target_idx = df_a[df_a["some_data"].isna()].index
for i, row in tqdm(df_a.loc[target_idx].iterrows()):
    df_a.at[i, "some_data"] = df_b.loc[df_b["user_id"] == row["user_id"], "alt_data"].iloc[0]
print(f"time check 2 ::: {time.time() - start}")

# 실험군 2 : merge 이용
# merge는 C 기반의 "조인 알고리즘"을 사용 -> 속도 빠름
# 파이썬 반복이 없음
# boolean filtering이 반복되지 않음
# 해시 기반 매칭으로 접근 속도가 O(1)에 가까움
# 전체 작업이 내부적으로 벡터화되어 있으며 C/Cython 코드로 동작
# DataFrame 블록 구조를 활용해 메모리 접근도 효율적
df_a = df_a_origin.copy()
start = time.time()
df_merge = df_a.merge(df_b, on="user_id", how="left")
df_merge["some_data"] = df_merge["some_data"].fillna(df_merge["alt_data"])
df_merge
print(f"time check 3 ::: {time.time() - start}")

0it [00:00, ?it/s]

10000it [00:01, 6217.42it/s]


time check 1 ::: 1.6103863716125488


1998it [00:01, 1469.15it/s]

time check 2 ::: 1.3619704246520996
time check 3 ::: 0.0045146942138671875
